In [2]:
"""
프로젝트: 여러 이미지를 합치는 프로그램을 만드시오

[사용자 시나리오]
1. 사용자는 합치려는 이미지를 1개 이상 선택한다.
2. 합쳐진 이미지가 저장될 경로를 지정한다.
3. 가로넓이 간격 포맷 옵션을 지정한다
4. 시작 버튼을 통해 이미지를 합친다.
5. 닫기 버튼을 통해 프로그램을 종료한다.

[기능 명세]
1. 파일추가: 리스트박스에 파일 추가
2. 선택삭제: 리스트 박스에서 선택된 항목 삭제
3. 찾아보기: 저장 폴더를 선택하면 텍스트 위젯에 입력
4. 가로넓이: 이미지 넓이 지정 (원본, 1024, 800, 640)
5. 간격: 이미지 간의 간격 지정 (없음, 좁게, 보통, 넓게)
6. 포맷: 저장 이미지 포맷 지정 (PNG, JPG, BMP)
7. 시작: 이미지 합치기 작업 실행
8. 진행상황: 현재 진행중인 파일 순서에 맞게 반영
9. 닫기: 프로그램 종료
"""

'\n프로젝트: 여러 이미지를 합치는 프로그램을 만드시오\n\n[사용자 시나리오]\n1. 사용자는 합치려는 이미지를 1개 이상 선택한다.\n2. 합쳐진 이미지가 저장될 경로를 지정한다.\n3. 가로넓이 간격 포맷 옵션을 지정한다\n4. 시작 버튼을 통해 이미지를 합친다.\n5. 닫기 버튼을 통해 프로그램을 종료한다.\n\n[기능 명세]\n1. 파일추가: 리스트박스에 파일 추가\n2. 선택삭제: 리스트 박스에서 선택된 항목 삭제\n3. 찾아보기: 저장 폴더를 선택하면 텍스트 위젯에 입력\n4. 가로넓이: 이미지 넓이 지정 (원본, 1024, 800, 640)\n5. 간격: 이미지 간의 간격 지정 (없음, 좁게, 보통, 넓게)\n6. 포맷: 저장 이미지 포맷 지정 (PNG, JPG, BMP)\n7. 시작: 이미지 합치기 작업 실행\n8. 진행상황: 현재 진행중인 파일 순서에 맞게 반영\n9. 닫기: 프로그램 종료\n'

In [16]:
from tkinter import *
from tkinter import filedialog
from PIL import Image
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox
import os


#레이아웃

root = Tk()
root.title("제목 없음 -Window 메모장")
root.resizable(False, False)

# 파일 프레임
file_frame=Frame(root)
file_frame.pack(fill="x", padx=5, pady=5)

def add_file(): # initialdir: 지정한 경로가 시작 위치
    files=filedialog.askopenfilenames(title="이미지 파일을 선택하세요", filetypes=(("PNG파일", "*.PNG"),("모든 파일", "*.*")), initialdir="H:\etc\Etc\Python")
    for file in files:
        listfile.insert(END, file)
        
def del_file():
    for index in reversed(listfile.curselection()): #reverse는 뒤집어서 저장, reversed는 저장 X
        listfile.delete(index)

def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected == "": #사용자가 취소를 누를 때, NONE 사용시 취소 누르면 경로가 지워짐
        print("폴더 선택 취소")
        return
    #print(folder_selected)
    txt_dest_path.delete(0, END)
    txt_dest_path.insert(0, folder_selected)
    
# 이미지 통합
def merge_image():
    # print("가로넓이 : ", cmb_width.get())
    # print("간격 : ", cmb_space.get())
    # print("포맷 : ", cmb_format.get())

    try:
        # 가로넓이
        img_width = cmb_width.get()
        if img_width == "원본 유지":
            img_width = -1 # -1 일때는 원본 기준으로
        else:
            img_width = int(img_width)

        # 간격
        img_space = cmb_space.get()
        if img_space == "좁게":
            img_space = 30
        elif img_space == "보통":
            img_space = 60
        elif img_space == "넓게":
            img_space = 90
        else: # 없음
            img_space = 0

        # 포맷
        img_format = cmb_format.get().lower() # PNG, JPG, BMP 값을 받아와서 소문자로 변경

        #####################################

        images = [Image.open(x) for x in listfile.get(0, END)]    

        # 이미지 사이즈 리스트에 넣어서 하나씩 처리
        image_sizes = [] # [(width1, height1), (width2, height2), ...]
        if img_width > -1:
            # width  값 변경
            image_sizes = [(int(img_width), int(img_width * x.size[1] / x.size[0])) for x in images]
        else:
            # 원본 사이즈 사용
            image_sizes = [(x.size[0], x.size[1]) for x in images]

        # 계산식
        # 100 * 60 이미지가 있음. -> width 를 80 으로 줄이면 height 는?
        # (원본 width) : (원본 height) = (변경 width) : (변경 height)
        #     100      :     60       =      80      :     ?
        #      x       :     y        =      x'      :     y'
        #    xy' = x'y
        #    y' = x'y / x -> 이 식을 적용
        #  100:60=80:48

        # 우리 코드에 대입하려면?
        # x = width = size[0]
        # y = height = size[1]
        # x' = img_width # 이 값으로 변경 해야 함
        # y' = x'y / x = img_width * size[1] / size[0]

        widths, heights = zip(*(image_sizes))

        # 최대 넓이, 전체 높이 구해옴
        max_width, total_height = max(widths), sum(heights)
        
        # 스케치북 준비
        if img_space > 0: # 이미지 간격 옵션 적용
            total_height += (img_space * (len(images) - 1))

        result_img = Image.new("RGB", (max_width, total_height), (255, 255, 255)) # 배경 흰색
        y_offset = 0 # y 위치

        for idx, img in enumerate(images):
            # width 가 원본유지가 아닐 때에는 이미지 크기 조정
            if img_width > -1:
                img = img.resize(image_sizes[idx])

            result_img.paste(img, (0, y_offset))
            y_offset += (img.size[1] + img_space) # height 값 + 사용자가 지정한 간격

            progress = (idx + 1) / len(images) * 100 # 실제 percent 정보를 계산
            p_var.set(progress)
            progressbar.update()

        # 포맷 옵션 처리
        file_name = "nado_photo." + img_format
        dest_path = os.path.join(txt_dest_path.get(), file_name)
        result_img.save(dest_path)
        msgbox.showinfo("알림", "작업이 완료되었습니다.")
    except Exception as err: # 예외처리 - 지정된 경로가 없거나, 파일을 사용할 수 없을 때
        msgbox.showerror("에러", err)

def start():
    #각 옵션의 값을 확인
    print("가로넓이: ",cmb_width.get())
    print("간격: ",cmb_space.get())
    print("포맷: ",cmb_format.get())
    
    # 파일 목록 확인
    if listfile.size()==0:
        msgbox.showwarning("경고, 아무것도 없음", "이미지 파일을 추가하세요")
        return
    
    #지정 경로 확인
    if len(txt_dest_path.get())==0:
        msgbox.showwarning("경고, 아무것도 없음", "저장 경로를 추가하세요")
        return
    
    # 이미지 통합
    merge_image()
    
btn_add_file = Button(file_frame, padx=5, pady=5,width=12, text="파일 추가", command=add_file).pack(side="left")
btn_del_file = Button(file_frame, padx=5, pady=5,width=12, text="선택 삭제", command=del_file).pack(side="right")

#리스트 프레임
listframe=Frame(root)
listframe.pack(fill="both")

scrollbar = Scrollbar(listframe)
scrollbar.pack(side="right", fill="y")

listfile=Listbox(listframe, selectmode="extended", height=15, yscrollcommand=scrollbar.set)
listfile.pack(side="left", fill="both", expand=True)
scrollbar.config(command=listfile.yview)

#저장 경로 프레임
pathframe = LabelFrame(root, text="저장 경로")
pathframe.pack(fill="x", padx=5, pady=5)

txt_dest_path = Entry(pathframe)
txt_dest_path.pack(side="left", fill="x", expand=True, padx=5, pady=5, ipady=4) #ipady 높이 변경

btn_dest_path = Button(pathframe, text="찾아보기", width = 10, command = browse_dest_path)
btn_dest_path.pack(side="right", padx=5, pady=5)

#옵션 프레임
optionframe = LabelFrame(root, text="옵션")
optionframe.pack(padx=5, pady=5)

#가로넓이 옵션
lbl_width = Label(optionframe, text="가로 넓이", width=8).pack(side="left")
#가로 넓이 콤보 박스
opt_width=["원본 유지", "1024", "800", "640"]
cmb_width = ttk.Combobox(optionframe, state="readonly", value=opt_width, width=10)
cmb_width.current(0)
cmb_width.pack(side="left", padx=5, pady=5)

#간격 옵션
lbl_space = Label(optionframe, text="간격", width=8).pack(side="left", padx=5, pady=5)
#간격 콤보 박스
opt_space=["없음", "좁게", "보통", "넓게"]
cmb_space = ttk.Combobox(optionframe, state="readonly",value=opt_space, width=10)
cmb_space.current(0)
cmb_space.pack(side="left", padx=5, pady=5)

#파일 포맷 옵션
lbl_format = Label(optionframe, text="포맷", width=8).pack(side="left", padx=5, pady=5)
#파일 포맷 콤보 박스
opt_format=["PNG", "JPG", "BMP"]
cmb_format = ttk.Combobox(optionframe, state="readonly",value=opt_format, width=10)
cmb_format.current(0)
cmb_format.pack(side="left", padx=5, pady=5)

#진행상황 프로그래스바
frameprogress = LabelFrame(root, text="진행상황", padx=5, pady=5)
frameprogress.pack(fill="x")

p_var = DoubleVar()
progressbar =ttk.Progressbar(frameprogress, maximum=100,variable=p_var)
progressbar.pack(fill="x", padx=5, pady=5)

#실행 프레임
framerun=Frame(root)
framerun.pack(fill="x")

btn_close = Button(framerun, padx=5, pady=5, text="닫기", width=12, command=root.destroy) #root.quit을 사용하면 오류가 생김
btn_close.pack(side="right", padx=5, pady=5)

btn_start = Button(framerun, padx=5, pady=5, text="시작", width=12, command=start)
btn_start.pack(side="right", padx=5, pady=5)

root.mainloop()

가로넓이:  원본 유지
간격:  보통
포맷:  BMP


In [17]:
#자동으로 스크린샷을 찍어주는 프로그램
from PIL import ImageGrab
import time

time.sleep(5)

for i in range(1, 10):
    img = ImageGrab.grab()
    img.save("image{}.png".format(i))
    time.sleep(2)
    

In [1]:
# 원할 때 스크린샷을 찍어주는 프로그램
import keyboard
import time
from PIL import ImageGrab

def screenshot():
    # 2023년 9월 1일 18시 08분 46초 - _20230901_180846
    curr_time = time.strftime("_%Y%m%d_%H%M%S")
    img = ImageGrab.grab()
    img.save("image{}.png".format(curr_time)) # 시간에 따라 파일명을 바꿔가면서 저장
    
keyboard.add_hotkey("F9", screenshot)
keyboard.wait("esc")

In [ ]:
# zip과 Unzip
kor = ["사과", "바나나", "오렌지"]
eng = ["apple", "banana", "orange"]
print(list(zip(kor, eng)))

mixed= [('사과', 'apple'), ('바나나', 'banana'), ('오렌지', 'orange')]
print(list(zip(*mixed)))

kor2, eng2 = zip(*mixed)
print(kor2, eng2)